## Viburnum RAD-seq demultiplexing notebook

This notebook contains the *ipyrad* code used to demultiplex fastq data from multiple RAD-seq libraries sequenced across multiple lanes of Illumina HiSeq, and to visualize the distribution of reads among samples. Information about the libraries is recorded below. This notebook and it's accompanying barcode files are archived online in a github repository [http://github.com/dereneaton/Viburnum-phylogeny](https://github.com/dereneaton/Viburnum-phylogeny). It may be updated as new data are attained. 

### Raw fastq reads and index (barcode) files
The libraries were prepared with *inline* barcodes that 10bp in length. Barcodes files that match sample names to barcodes are used to demultiplex the data. These files are available online, and used below. 
1. [Viburnum library-1 barcodes](https://github.com/dereneaton/Viburnum-phylogeny/blob/master/VIBURNUM_1_BARCODES.txt)
2. [Viburnum library-2 barcodes](https://github.com/dereneaton/Viburnum-phylogeny/blob/master/VIBURNUM_2_BARCODES.txt)  
3. [Viburnum library-3 barcodes](https://github.com/dereneaton/Viburnum-phylogeny/blob/master/VIBURNUM_3_BARCODES.txt)  
3. [Viburnum library-4 barcodes](https://github.com/dereneaton/Viburnum-phylogeny/blob/master/VIBURNUM_4_BARCODES.txt)  


### The local paths to raw data files
All of these libraries were sequenced twice, on two separate lanes, to increase read depths. We could simply read in all of the data from both lanes, along with their corresponding barcodes file, to demultiplex each library. However, the more proper way to do things is to demultiplex each lane of data separately so that any possible effect of the lane on our results could (at least in theory) be detected. This is also the recommended format for uploading data to the NCBI short read archive, with each lane of data separated into separate files. It's really not that much trouble, since later when we go to assemble the data reads from the same sample, but which were demultiplexed into separate files, representing the separate lanes, can be easily merged using the `ipyrad.merge()` command.  

In [12]:
## Name: Viburnum 1
## Description: mostly species-level sampling for phylogeny but also some
##              population-level sampling for Beth's thesis, including
##              nudum, dentatum, rufidulum, lentago, prunifolium.
## Sequencer: 2 lanes; 100bp; SE; Illumina Hi-Seq 2000 at Univ. Oregon
## Lib-prep: Floragenex, PstI enzyme, size-selection:?

## there are two compressed data files (2 lanes), each 16Gb in size.
lib1_1 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_1/UO_C353_*.gz"
lib1_2 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_1/UO_C354_*.gz"
bar1 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/barcodes/VIBURNUM_1_BARCODES.txt"

In [13]:
## Name: Viburnum 2
## Description: mostly species-level sampling for phylogeny but also some
##              population-level sampling for Beth's thesis, including
##              lentago, prunifolium, rufidulum, obovatum.
## Sequencer: 2 lanes; 100bp; SE; Illumina Hi-Seq 2000 and 2500 at Univ. Oregon
## Lib-prep: Floragenex, PstI enzyme, size-selection: ?

## The first lane is HiSeq 2000, 9 files each ~1.4Gb in size.
## The second lane is HiSeq 2500, 7 files each ~1.4Gb in size.
lib2_1 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_2/lane1_*.gz"
lib2_2 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_2/lane8_*.gz"
bar2 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/barcodes/VIBURNUM_2_BARCODES.txt"

In [14]:
## Name: Viburnum 3
## Description: dentatum & nudum sampling for Beth's thesis
## Sequencer: 2 lanes; 100bp; SE; Illumina Hi-Seq 2500 at Univ. Oregon
## Lib-prep: Floragenex, PstI enzyme, size-selection:?

## there are two compressed data files (2 lanes), each 9Gb in size.
lib3_1 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_3/261_*.gz"
lib3_2 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_3/262_*.gz"
bar3 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/barcodes/VIBURNUM_3_BARCODES.txt"

In [15]:
## Name: Viburnum 4
## Description: mostly dentatum & rufidulum sampling for Beth's thesis
## Sequencer: 2 lanes; 100bp; SE; Illumina Hi-Seq 2500 at Univ. Oregon
## Lib-prep: Floragenex, PstI enzyme, size-selection:?

## there are two compressed data files (2 lanes), each 11Gb in size.
lib4_1 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_4/263_*.gz"
lib4_2 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/VIBURNUM_4/264_*.gz"
bar4 = "/ysm-gpfs/project/de243/RADSEQ_RAWS/barcodes/VIBURNUM_4_BARCODES.txt"

### Load *ipyrad* and print the current version

In [16]:
import ipyrad as ip
print "ipyrad v.{}".format(ip.__version__)

ipyrad v.0.5.10


### Initiate an Assembly class objects for each lane of data


In [17]:
## create Assemblies for each library
data1_1 = ip.Assembly("lib1-lane1")
data1_2 = ip.Assembly("lib1-lane2")
data2_1 = ip.Assembly("lib2-lane1")
data2_2 = ip.Assembly("lib2-lane2")
data3_1 = ip.Assembly("lib3-lane1")
data3_2 = ip.Assembly("lib3-lane2")
data4_1 = ip.Assembly("lib4-lane1")
data4_2 = ip.Assembly("lib4-lane2")

  New Assembly: lib1-lane1
  New Assembly: lib1-lane2
  New Assembly: lib2-lane1
  New Assembly: lib2-lane2
  New Assembly: lib3-lane1
  New Assembly: lib3-lane2
  New Assembly: lib4-lane1
  New Assembly: lib4-lane2


### Set parameters for each Assembly

Here we set the path to the data files, as well as set a common project directory for all of the assemblies so that the resulting files that they produce will all be grouped into one place. Because the barcodes are 10bp in length we will allow 1 bp error in barcodes during demultiplexing. 

In [18]:
## Path where we want to write all of the demux files 
demuxdir = "/ysm-gpfs/project/de243/Viburnum_demux"

## set data & barcodes paths for each library
data1_1.set_params("project_dir", demuxdir)
data1_1.set_params("raw_fastq_path", lib1_1)
data1_1.set_params("barcodes_path", bar1)
data1_1.set_params("max_barcode_mismatch", 1)

## set data & barcodes paths for each library
data1_2.set_params("project_dir", demuxdir)
data1_2.set_params("raw_fastq_path", lib1_2)
data1_2.set_params("barcodes_path", bar1)
data1_2.set_params("max_barcode_mismatch", 1)


## set data & barcodes paths for each library
data2_1.set_params("project_dir", demuxdir)
data2_1.set_params("raw_fastq_path", lib2_1)
data2_1.set_params("barcodes_path", bar2)
data2_1.set_params("max_barcode_mismatch", 1)

## set data & barcodes paths for each library
data2_2.set_params("project_dir", demuxdir)
data2_2.set_params("raw_fastq_path", lib2_2)
data2_2.set_params("barcodes_path", bar2)
data2_2.set_params("max_barcode_mismatch", 1)


## set data & barcodes paths for each library
data3_1.set_params("project_dir", demuxdir)
data3_1.set_params("raw_fastq_path", lib3_1)
data3_1.set_params("barcodes_path", bar3)
data3_1.set_params("max_barcode_mismatch", 1)

## set data & barcodes paths for each library
data3_2.set_params("project_dir", demuxdir)
data3_2.set_params("raw_fastq_path", lib3_2)
data3_2.set_params("barcodes_path", bar3)
data3_2.set_params("max_barcode_mismatch", 1)


## set data & barcodes paths for each library
data4_1.set_params("project_dir", demuxdir)
data4_1.set_params("raw_fastq_path", lib4_1)
data4_1.set_params("barcodes_path", bar4)
data4_1.set_params("max_barcode_mismatch", 1)

## set data & barcodes paths for each library
data4_2.set_params("project_dir", demuxdir)
data4_2.set_params("raw_fastq_path", lib4_2)
data4_2.set_params("barcodes_path", bar4)
data4_2.set_params("max_barcode_mismatch", 1)


In [20]:
## run demultiplexing (step 1) for each Assembly
data1_1.run("1")
data1_2.run("1")


  Assembly: lib1-lane1
  [####################] 100%  chunking large files  | 0:07:26 | s1 | 
  [####################] 100%  sorting reads         | 0:15:38 | s1 | 
  [####################] 100%  writing/compressing   | 0:23:19 | s1 | 

  Assembly: lib1-lane2
  [####################] 100%  chunking large files  | 0:07:32 | s1 | 
  [####################] 100%  sorting reads         | 0:15:33 | s1 | 
  [####################] 100%  writing/compressing   | 0:20:33 | s1 | 


In [21]:
data2_1.run("1")
data2_2.run("1")


  Assembly: lib2-lane1
  [####################] 100%  chunking large files  | 0:06:38 | s1 | 
  [####################] 100%  sorting reads         | 0:12:39 | s1 | 
  [####################] 100%  writing/compressing   | 0:15:37 | s1 | 

  Assembly: lib2-lane2
  [####################] 100%  chunking large files  | 0:04:58 | s1 | 
  [####################] 100%  sorting reads         | 0:09:27 | s1 | 
  [####################] 100%  writing/compressing   | 0:10:47 | s1 | 


In [22]:
data3_1.run("1")
data3_2.run("1")


  Assembly: lib3-lane1
  [####################] 100%  chunking large files  | 0:04:31 | s1 | 
  [####################] 100%  sorting reads         | 0:10:06 | s1 | 
  [####################] 100%  writing/compressing   | 0:14:36 | s1 | 

  Assembly: lib3-lane2
  [####################] 100%  chunking large files  | 0:04:31 | s1 | 
  [####################] 100%  sorting reads         | 0:09:58 | s1 | 
  [####################] 100%  writing/compressing   | 0:12:28 | s1 | 


In [23]:
data4_1.run("1")
data4_2.run("1")


  Assembly: lib4-lane1
  [####################] 100%  chunking large files  | 0:05:14 | s1 | 
  [####################] 100%  sorting reads         | 0:11:42 | s1 | 
  [####################] 100%  writing/compressing   | 0:16:21 | s1 | 

  Assembly: lib4-lane2
  [####################] 100%  chunking large files  | 0:05:05 | s1 | 
  [####################] 100%  sorting reads         | 0:11:22 | s1 | 
  [####################] 100%  writing/compressing   | 0:16:13 | s1 | 


### File access permissions
Because I want all users in my group to be able to access the new demux folder that we created somewhere in our cluster directory, I will use the unix command chmod to make it accessible to users in my 'group'. 

In [24]:
## the (!) means this is a bash command, 
## chmod changes the permissions, 
## the order is: me=7, group=7, others=4
## -R means apply to subfolders as well
! chmod -R 774 $demuxdir


## Analysis/Visualization of read distributions

In [122]:
## import some plotting libraries
import toyplot
import pandas as pd

### Summary of reads 

In [124]:
## a quick summary of the raw_reads stats
raws = pd.DataFrame([
    data1_1.stats.reads_raw.describe(),
    data1_2.stats.reads_raw.describe(),
    data2_1.stats.reads_raw.describe(),
    data2_2.stats.reads_raw.describe(),
    data3_1.stats.reads_raw.describe(),
    data3_2.stats.reads_raw.describe(),
    data4_1.stats.reads_raw.describe(),
    data4_2.stats.reads_raw.describe(),    
        ], index=["data1_1", "data1_2", "data2_1", "data2_2", 
                  "data3_1", "data3_2", "data4_1", "data4_2"]
                   )

print raws.astype(int)

         count     mean      std     min      25%      50%      75%       max
data1_1     96  1716711  1177834  443002  1133123  1453248  1958062  10507351
data1_2     96  1720723  1183210  437908  1138123  1473712  1979297  10704173
data2_1     96  1277771   926108  318767   889236  1060515  1403085   8539364
data2_2     96   938523   694224  233855   650632   781815  1040947   6440822
data3_1     96  1098159   638534  399155   885652   995892  1172098   6595603
data3_2     96  1075994   588550  432182   840792  1019919  1137892   6090308
data4_1     96  1276536   904701  138413   691539  1133506  1580125   5196314
data4_2     96  1238458   867018  139396   664317  1076430  1544365   4949833


### Replicate lanes
The plots below show the number of reads for each sample in a library, ordered from lowest to highest, with the two replicates lanes of each library shown in different colors. As you can see the replicate lanes usually recovered almost identical numbers of reads. 

In [123]:
canvas = toyplot.Canvas(width=800, height=600)

## plot lib1
axes = canvas.cartesian(label="lib1", grid=(2, 2, 0, 0), 
                        yscale='log', ymin=int(1e5), ymax=int(1e7))
axes.scatterplot(sorted(data1_1.stats.reads_raw), opacity=0.4, size=10)
axes.scatterplot(sorted(data1_2.stats.reads_raw), opacity=0.8)
axes.x.ticks.show = True
axes.y.ticks.show = True

## plot lib2
axes = canvas.cartesian(label="lib2", grid=(2, 2, 0, 1), 
                        yscale='log', ymin=int(1e5), ymax=int(1e7))
axes.scatterplot(sorted(data2_1.stats.reads_raw), opacity=0.4, size=10)
axes.scatterplot(sorted(data2_2.stats.reads_raw), opacity=0.8)
axes.x.ticks.show = True
axes.y.ticks.show = True

## plot lib3
axes = canvas.cartesian(label="lib3", grid=(2, 2, 1, 0), 
                        yscale='log', ymin=int(1e5), ymax=int(1e7))
axes.scatterplot(sorted(data3_1.stats.reads_raw), opacity=0.4, size=10)
axes.scatterplot(sorted(data3_2.stats.reads_raw), opacity=0.8)
axes.x.ticks.show = True
axes.y.ticks.show = True

## plot lib4
axes = canvas.cartesian(label="lib4", grid=(2, 2, 1, 1), 
                        yscale='log', ymin=int(1e5), ymax=int(1e7))
axes.scatterplot(sorted(data4_1.stats.reads_raw), opacity=0.4, size=10)
axes.scatterplot(sorted(data4_2.stats.reads_raw), opacity=0.8)
axes.x.ticks.show = True
axes.y.ticks.show = True

0 30 60 90 10 5 10 6 10 7 lib1 0 30 60 90 10 5 10 6 10 7 lib2 0 30 60 90 10 5 10 6 10 7 lib3 0 30 60 90 10 5 10 6 10 7 lib4 
 
 
 Save as .csv